## Data Analysis - Goal: to join supply and demand information about diesel

In [35]:
import pandas as pd
import numpy as np

Read the csv files that were downloaded in the ANP web site

In [36]:
# To connect to the csv file that is downlaoded in the ANP web site
# https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/processamento-de-petroleo-e-producao-de-derivados
# https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/vendas-de-derivados-de-petroleo-e-biocombustiveis
demand = pd.read_csv(
    'vendas-combustiveis-segmento-m3-2012-2025.csv', 
    sep=';',
    decimal=',',
    thousands='.'
    )
supply = pd.read_csv(
    'producao-derivados-petroleo-por-refinaria-m3-1990-2025.csv',
    sep=';',
    decimal=',',
    thousands='.'
)

In [37]:
start_year = 2012
end_year = 2024

In [38]:
print(demand.head())
print(demand.tail())

    ANO  MÊS UNIDADE DA FEDERAÇÃO           PRODUTO          SEGMENTO  VENDAS
0  2012  JAN             RONDÔNIA  ETANOL HIDRATADO  POSTO REVENDEDOR  1560.5
1  2012  MAI     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    50.0
2  2012  JUN     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    25.0
3  2012  JUL     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    25.0
4  2012  AGO     DISTRITO FEDERAL  ETANOL HIDRATADO  CONSUMIDOR FINAL    20.0
        ANO  MÊS UNIDADE DA FEDERAÇÃO      PRODUTO          SEGMENTO  \
38146  2025  JAN   MATO GROSSO DO SUL  ÓLEO DIESEL  POSTO REVENDEDOR   
38147  2025  JAN    RIO GRANDE DO SUL  ÓLEO DIESEL  POSTO REVENDEDOR   
38148  2025  JAN       SANTA CATARINA  ÓLEO DIESEL  POSTO REVENDEDOR   
38149  2025  JAN            SÃO PAULO  ÓLEO DIESEL  POSTO REVENDEDOR   
38150  2025  JAN     DISTRITO FEDERAL  ÓLEO DIESEL               TRR   

           VENDAS  
38146   71057.868  
38147  206235.406  
38148  185694.560  
38149  532111.030  

In [39]:
print(supply.head())
print(supply.tail())

    ANO  MÊS UNIDADE DA FEDERAÇÃO REFINARIA              PRODUTO  PRODUÇÃO
0  1990  JAN                CEARÁ    LUBNOR             PARAFINA       0.0
1  1990  MAI            SÃO PAULO      RPBC  GASOLINA DE AVIAÇÃO   23832.0
2  1990  JUN            SÃO PAULO      RPBC  GASOLINA DE AVIAÇÃO       0.0
3  1990  JUL            SÃO PAULO      RPBC  GASOLINA DE AVIAÇÃO       0.0
4  1990  AGO            SÃO PAULO      RPBC  GASOLINA DE AVIAÇÃO    3995.0
         ANO  MÊS UNIDADE DA FEDERAÇÃO REFINARIA               PRODUTO  \
109969  2025  JAN            SÃO PAULO    UNIVEN  QUEROSENE DE AVIAÇÃO   
109970  2025  FEV    RIO GRANDE DO SUL     REFAP  QUEROSENE DE AVIAÇÃO   
109971  2025  JAN    RIO GRANDE DO SUL     REFAP  QUEROSENE DE AVIAÇÃO   
109972  2025  FEV                BAHIA    REFMAT  QUEROSENE ILUMINANTE   
109973  2025  JAN            SÃO PAULO     SSOIL    OUTROS ENERGÉTICOS   

        PRODUÇÃO  
109969     0.000  
109970  9661.907  
109971  9564.305  
109972   188.100  
109973    

In [40]:
# demand_filter_product = demand[(demand['PRODUTO'] == 'GASOLINA C') | (demand['PRODUTO'] == 'ETANOL HIDRATADO')]

# Filter only the lines which 'PRODUTO' is equal to GASOLINA C
demand_filter_product = demand[(demand['PRODUTO'] == 'ÓLEO DIESEL')]

In [41]:
# Create a columns for the number month
month_numbers = {'JAN' : '01',
                 'FEV' : '02',
                 'MAR' : '03',
                 'ABR' : '04',
                 'MAI' : '05',
                 'JUN' : '06',
                 'JUL' : '07',
                 'AGO' : '08',
                 'SET' : '09',
                 'OUT' : '10',
                 'NOV' : '11',
                 'DEZ' : '12',
                 }
demand_filter_product = demand_filter_product.copy()
demand_filter_product['MONTH'] = demand_filter_product['MÊS'].map(month_numbers)
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH
1944,2012,JAN,RONDÔNIA,ÓLEO DIESEL,POSTO REVENDEDOR,35384.62523,01
1945,2012,MAI,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14276.30000,05
1946,2012,JUN,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,13238.10000,06
1947,2012,JUL,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14116.10000,07
1948,2012,AGO,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,15291.75300,08


In [42]:
demand_filter_product = demand_filter_product.copy()
demand_filter_product['Date'] = pd.to_datetime(demand_filter_product['ANO'].astype(str) + '-' + demand_filter_product['MONTH'].astype(str) + '-01')

# ethanol_blend = {
#     2012: 20.0,
#     2013: 25.0,
#     2014: 25.0,
#     2015: 27.0,
#     2016: 27.0,
#     2017: 27.0,
#     2018: 27.0,
#     2019: 27.0,
#     2020: 27.0,
#     2021: 27.0,
#     2022: 27.0,
#     2023: 27.5,
#     2024: 27.5
# }


# demand_filter_product['ethanol_blend'] = demand_filter_product['ANO'].map(ethanol_blend)
# demand_filter_product['VENDAS_GASOLINA_A'] = (1-demand_filter_product['ethanol_blend']/100)*demand_filter_product['VENDAS']
# demand_filter_product.head()



In [43]:
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date
1944,2012,JAN,RONDÔNIA,ÓLEO DIESEL,POSTO REVENDEDOR,35384.62523,01,2012-01-01
1945,2012,MAI,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14276.30000,05,2012-05-01
1946,2012,JUN,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,13238.10000,06,2012-06-01
1947,2012,JUL,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14116.10000,07,2012-07-01
1948,2012,AGO,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,15291.75300,08,2012-08-01


In [44]:
demand_filter_product.tail()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date
38146,2025,JAN,MATO GROSSO DO SUL,ÓLEO DIESEL,POSTO REVENDEDOR,71057.868,01,2025-01-01
38147,2025,JAN,RIO GRANDE DO SUL,ÓLEO DIESEL,POSTO REVENDEDOR,206235.406,01,2025-01-01
38148,2025,JAN,SANTA CATARINA,ÓLEO DIESEL,POSTO REVENDEDOR,185694.560,01,2025-01-01
38149,2025,JAN,SÃO PAULO,ÓLEO DIESEL,POSTO REVENDEDOR,532111.030,01,2025-01-01
38150,2025,JAN,DISTRITO FEDERAL,ÓLEO DIESEL,TRR,535.000,01,2025-01-01


In [45]:
federative_unit = {
    'ACRE' : 'AC',
    'ALAGOAS' : 'AL',
    'AMAPÁ' : 'AP',
    'AMAZONAS' : 'AM',
    'BAHIA' : 'BA',
    'CEARÁ' : 'CE',
    'DISTRITO FEDERAL' : 'DF',
    'ESPÍRITO SANTO' : 'ES',
    'GOIÁS' : 'GO',
    'MARANHÃO' : 'MA',
    'MATO GROSSO' : 'MT',
    'MATO GROSSO DO SUL' : 'MS',
    'MINAS GERAIS' : 'MG',
    'PARANÁ' : 'PR',
    'PARAÍBA' : 'PB',
    'PARÁ' : 'PA',
    'PARA' : 'PA',
    'PERNAMBUCO' : 'PE',
    'PIAUÍ' : 'PI',
    'RIO DE JANEIRO' : 'RJ',
    'RIO GRANDE DO SUL' : 'RS',
    'RIO GRANDE DO NORTE' : 'RN',
    'RONDÔNIA' : 'RO',
    'RONDONIA' : 'RO',
    'RORAIMA' : 'RR',
    'SANTA CATARINA' : 'SC',
    'SERGIPE' : 'SE',
    'SÃO PAULO' : 'SP',
    'TOCANTINS' : 'TO'
}
demand_filter_product = demand_filter_product.copy()
demand_filter_product['UF'] = demand_filter_product['UNIDADE DA FEDERAÇÃO'].map(federative_unit)
demand_filter_product.head()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date,UF
1944,2012,JAN,RONDÔNIA,ÓLEO DIESEL,POSTO REVENDEDOR,35384.62523,01,2012-01-01,RO
1945,2012,MAI,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14276.30000,05,2012-05-01,DF
1946,2012,JUN,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,13238.10000,06,2012-06-01,DF
1947,2012,JUL,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,14116.10000,07,2012-07-01,DF
1948,2012,AGO,DISTRITO FEDERAL,ÓLEO DIESEL,CONSUMIDOR FINAL,15291.75300,08,2012-08-01,DF


In [46]:
demand_filter_product_time = demand_filter_product.copy()
demand_filter_product_time = demand_filter_product_time[(demand_filter_product_time['ANO'] >= start_year) & (demand_filter_product_time['ANO'] <= end_year)]
demand_filter_product_time.tail()

,ANO,MÊS,UNIDADE DA FEDERAÇÃO,PRODUTO,SEGMENTO,VENDAS,MONTH,Date,UF
37903,2024,DEZ,DISTRITO FEDERAL,ÓLEO DIESEL,POSTO REVENDEDOR,16938.687,12,2024-12-01,DF
37904,2024,NOV,DISTRITO FEDERAL,ÓLEO DIESEL,POSTO REVENDEDOR,16987.500,11,2024-11-01,DF
37905,2024,OUT,DISTRITO FEDERAL,ÓLEO DIESEL,POSTO REVENDEDOR,18830.200,10,2024-10-01,DF
37906,2024,SET,DISTRITO FEDERAL,ÓLEO DIESEL,POSTO REVENDEDOR,17706.026,09,2024-09-01,DF
37907,2024,FEV,RONDÔNIA,ÓLEO DIESEL,CONSUMIDOR FINAL,13974.846,02,2024-02-01,RO


In [48]:
demand_filter_product_time

first_group_demand = demand_filter_product_time.groupby(['Date', 'UF', 'PRODUTO'])['VENDAS'].sum().reset_index()
first_group_demand = first_group_demand.copy()
# first_group_demand['KEY'] = first_group_demand['Date'].astype(str) + '-' + first_group_demand['UF']
# first_group_demand['Date'].astype(str) + '-' +
#                             first_group_demand[]

first_group_demand

,Date,UF,PRODUTO,VENDAS
0,2012-01-01,AC,ÓLEO DIESEL,17186.421285
1,2012-01-01,AL,ÓLEO DIESEL,40867.282810
2,2012-01-01,AM,ÓLEO DIESEL,98427.858660
3,2012-01-01,AP,ÓLEO DIESEL,32762.135571
4,2012-01-01,BA,ÓLEO DIESEL,240449.988400
...,...,...,...,...
4207,2024-12-01,RS,ÓLEO DIESEL,323716.343000
4208,2024-12-01,SC,ÓLEO DIESEL,238956.852000
4209,2024-12-01,SE,ÓLEO DIESEL,33864.083000
4210,2024-12-01,SP,ÓLEO DIESEL,933532.537000


In [ ]:
supply = supply.copy() # Create a copy of the dataframe to avoid modifying the original data
supply = supply[(supply['ANO'] >= start_year) & (supply['ANO'] <= end_year)] # Filter the dataframe for the analysis period based on the start and end year inputs  
supply['UF'] = supply['UNIDADE DA FEDERAÇÃO'].map(federative_unit) # Map federal unit names to their corresponding abbreviations (UF)  
supply['MONTH'] = supply['MÊS'].map(month_numbers) # Convert month names to their corresponding numerical values  
supply['Date'] = pd.to_datetime(supply['ANO'].astype(str) + '-' + supply['MONTH'].astype(str) + '-01') # Create a datetime column using year and month (set day as 1 for consistency)  
# supply['ethanol_blend'] = supply['ANO'].map(ethanol_blend)
# supply['PRODUÇÃO_GAS_C'] = (100-supply['ethanol_blend'])/ 100 * supply['PRODUÇÃO'] 
# supply['PRODUÇÃO_GAS_C'] = (1-supply['ethanol_blend'])/ 100 * supply['PRODUÇÃO'] 

first_group_supply = supply.groupby(['Date', 'UF', 'REFINARIA', 'PRODUTO'])['PRODUÇÃO'].sum().reset_index() # Group by Date, UF, and PRODUTO, then sum the PRODUÇÃO values  
first_group_supply = first_group_supply[first_group_supply['PRODUTO'] == 'GASOLINA A'] # Filter only the lines which the values is 'GASOLINA A'
second_group_suply = first_group_supply.groupby(['Date', 'UF', 'PRODUTO'])['PRODUÇÃO'].sum().reset_index()
first_group_supply


,Date,UF,REFINARIA,PRODUTO,PRODUÇÃO
2,2012-01-01,AM,REAM,GASOLINA A,39165.000
17,2012-01-01,BA,DAX OIL,GASOLINA A,0.000
32,2012-01-01,BA,REFMAT,GASOLINA A,229124.000
47,2012-01-01,CE,LUBNOR,GASOLINA A,0.000
62,2012-01-01,MG,REGAP,GASOLINA A,143835.000
...,...,...,...,...,...
41884,2024-12-01,SP,RECAP,GASOLINA A,91477.671
41899,2024-12-01,SP,REPLAN,GASOLINA A,535003.300
41914,2024-12-01,SP,REVAP,GASOLINA A,293101.510
41929,2024-12-01,SP,RPBC,GASOLINA A,251148.080


In [ ]:

# Filter for gasoline products only 
supply_SandD_gas = second_group_suply[
    second_group_suply['PRODUTO'].isin(['GASOLINA A'])
].copy()  # Create independent copy to avoid warnings

# Create unique composite key: Date-UF-PRODUTO
supply_SandD_gas['KEY'] = (
    supply_SandD_gas['Date'].astype(str) + '-' + 
    supply_SandD_gas['UF'] ## + '-' + 
    ## demand_SandD_gas['PRODUTO']
)
# supply_SandD_gas['PRODUTO'] = supply_SandD_gas['PRODUTO'].replace(
#     {'ETANOL HIDRATADO': 'HIDRATADO'}
# )
supply_SandD_gas

,Date,UF,PRODUTO,PRODUÇÃO,KEY
0,2012-01-01,AM,GASOLINA A,39165.000,2012-01-01-AM
1,2012-01-01,BA,GASOLINA A,229124.000,2012-01-01-BA
2,2012-01-01,CE,GASOLINA A,0.000,2012-01-01-CE
3,2012-01-01,MG,GASOLINA A,143835.000,2012-01-01-MG
4,2012-01-01,PE,GASOLINA A,0.000,2012-01-01-PE
...,...,...,...,...,...
1553,2024-12-01,PE,GASOLINA A,2115.982,2024-12-01-PE
1554,2024-12-01,PR,GASOLINA A,349793.695,2024-12-01-PR
1555,2024-12-01,RJ,GASOLINA A,374071.843,2024-12-01-RJ
1556,2024-12-01,RS,GASOLINA A,284207.507,2024-12-01-RS


In [ ]:
demand_SandD_gas = first_group_demand.copy()  # Create independent copy to avoid warnings

# Create unique composite key: Date-UF-PRODUTO
demand_SandD_gas['KEY'] = (
    demand_SandD_gas['Date'].astype(str) + '-' + 
    demand_SandD_gas['UF'] ## + '-' + 
    ## demand_SandD_gas['PRODUTO']
)
# supply_SandD_gas['PRODUTO'] = supply_SandD_gas['PRODUTO'].replace(
#     {'ETANOL HIDRATADO': 'HIDRATADO'}
# )
demand_SandD_gas

,Date,UF,PRODUTO,VENDAS_GASOLINA_A,KEY
0,2012-01-01,AC,ÓLEO DIESEL,13749.137028,2012-01-01-AC
1,2012-01-01,AL,ÓLEO DIESEL,32693.826248,2012-01-01-AL
2,2012-01-01,AM,ÓLEO DIESEL,78742.286928,2012-01-01-AM
3,2012-01-01,AP,ÓLEO DIESEL,26209.708457,2012-01-01-AP
4,2012-01-01,BA,ÓLEO DIESEL,192359.990720,2012-01-01-BA
...,...,...,...,...,...
4207,2024-12-01,RS,ÓLEO DIESEL,234694.348675,2024-12-01-RS
4208,2024-12-01,SC,ÓLEO DIESEL,173243.717700,2024-12-01-SC
4209,2024-12-01,SE,ÓLEO DIESEL,24551.460175,2024-12-01-SE
4210,2024-12-01,SP,ÓLEO DIESEL,676811.089325,2024-12-01-SP


In [ ]:
supply_and_demand_SandD_gas = demand_SandD_gas.merge(
    supply_SandD_gas, 
    how='outer',
    on='KEY',
    suffixes=('_d', '_s')
    )
supply_and_demand_SandD_gas

,Date_d,UF_d,PRODUTO_d,VENDAS_GASOLINA_A,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
0,2012-01-01,AC,ÓLEO DIESEL,13749.137028,2012-01-01-AC,NaT,NaN,NaN,NaN
1,2012-01-01,AL,ÓLEO DIESEL,32693.826248,2012-01-01-AL,NaT,NaN,NaN,NaN
2,2012-01-01,AM,ÓLEO DIESEL,78742.286928,2012-01-01-AM,2012-01-01,AM,GASOLINA A,39165.000
3,2012-01-01,AP,ÓLEO DIESEL,26209.708457,2012-01-01-AP,NaT,NaN,NaN,NaN
4,2012-01-01,BA,ÓLEO DIESEL,192359.990720,2012-01-01-BA,2012-01-01,BA,GASOLINA A,229124.000
...,...,...,...,...,...,...,...,...,...
4207,2024-12-01,RS,ÓLEO DIESEL,234694.348675,2024-12-01-RS,2024-12-01,RS,GASOLINA A,284207.507
4208,2024-12-01,SC,ÓLEO DIESEL,173243.717700,2024-12-01-SC,NaT,NaN,NaN,NaN
4209,2024-12-01,SE,ÓLEO DIESEL,24551.460175,2024-12-01-SE,NaT,NaN,NaN,NaN
4210,2024-12-01,SP,ÓLEO DIESEL,676811.089325,2024-12-01-SP,2024-12-01,SP,GASOLINA A,1228801.568


In [ ]:
# First fill UF_s and UF_d from whichever column exists
supply_and_demand_SandD_gas['UF_d'] = supply_and_demand_SandD_gas['UF_d'].fillna(supply_and_demand_SandD_gas['UF_s'])
supply_and_demand_SandD_gas['UF_s'] = supply_and_demand_SandD_gas['UF_s'].fillna(supply_and_demand_SandD_gas['UF_d'])
supply_and_demand_SandD_gas['PRODUTO_s'] = supply_and_demand_SandD_gas['PRODUTO_s'].fillna(supply_and_demand_SandD_gas['PRODUTO_d'])
supply_and_demand_SandD_gas['PRODUTO_d'] = supply_and_demand_SandD_gas['PRODUTO_d'].fillna(supply_and_demand_SandD_gas['PRODUTO_s'])
supply_and_demand_SandD_gas['Date_s'] = supply_and_demand_SandD_gas['Date_s'].fillna(supply_and_demand_SandD_gas['Date_d'])
supply_and_demand_SandD_gas['Date_d'] = supply_and_demand_SandD_gas['Date_d'].fillna(supply_and_demand_SandD_gas['Date_s'])

# Then fill all remaining numeric columns with 0
numeric_cols = ['VENDAS_GASOLINA_A', 'PRODUÇÃO']
supply_and_demand_SandD_gas[numeric_cols] = supply_and_demand_SandD_gas[numeric_cols].fillna(0)

# For date columns, you might want to keep as NaT or fill with a default date
# supply_and_demand_SandD_ethanol['Date_d'] = supply_and_demand_SandD_ethanol['Date_d'].fillna(pd.NaT)
# supply_and_demand_SandD_ethanol['Date_s'] = supply_and_demand_SandD_ethanol['Date_s'].fillna(pd.NaT)
supply_and_demand_SandD_gas.head(10)

,Date_d,UF_d,PRODUTO_d,VENDAS_GASOLINA_A,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
0,2012-01-01,AC,ÓLEO DIESEL,13749.137028,2012-01-01-AC,2012-01-01,AC,ÓLEO DIESEL,0.0
1,2012-01-01,AL,ÓLEO DIESEL,32693.826248,2012-01-01-AL,2012-01-01,AL,ÓLEO DIESEL,0.0
2,2012-01-01,AM,ÓLEO DIESEL,78742.286928,2012-01-01-AM,2012-01-01,AM,GASOLINA A,39165.0
3,2012-01-01,AP,ÓLEO DIESEL,26209.708457,2012-01-01-AP,2012-01-01,AP,ÓLEO DIESEL,0.0
4,2012-01-01,BA,ÓLEO DIESEL,192359.990720,2012-01-01-BA,2012-01-01,BA,GASOLINA A,229124.0
5,2012-01-01,CE,ÓLEO DIESEL,63206.833760,2012-01-01-CE,2012-01-01,CE,GASOLINA A,0.0
6,2012-01-01,DF,ÓLEO DIESEL,22262.499016,2012-01-01-DF,2012-01-01,DF,ÓLEO DIESEL,0.0
7,2012-01-01,ES,ÓLEO DIESEL,73627.786264,2012-01-01-ES,2012-01-01,ES,ÓLEO DIESEL,0.0
8,2012-01-01,GO,ÓLEO DIESEL,123275.836480,2012-01-01-GO,2012-01-01,GO,ÓLEO DIESEL,0.0
9,2012-01-01,MA,ÓLEO DIESEL,74226.313936,2012-01-01-MA,2012-01-01,MA,ÓLEO DIESEL,0.0


In [ ]:
supply_and_demand_SandD_gas.tail(20)

,Date_d,UF_d,PRODUTO_d,VENDAS_GASOLINA_A,KEY,Date_s,UF_s,PRODUTO_s,PRODUÇÃO
4192,2024-12-01,ES,ÓLEO DIESEL,78429.043475,2024-12-01-ES,2024-12-01,ES,ÓLEO DIESEL,0.000
4193,2024-12-01,GO,ÓLEO DIESEL,174845.534150,2024-12-01-GO,2024-12-01,GO,ÓLEO DIESEL,0.000
4194,2024-12-01,MA,ÓLEO DIESEL,103167.929200,2024-12-01-MA,2024-12-01,MA,ÓLEO DIESEL,0.000
4195,2024-12-01,MG,ÓLEO DIESEL,460760.505700,2024-12-01-MG,2024-12-01,MG,GASOLINA A,210016.297
4196,2024-12-01,MS,ÓLEO DIESEL,92935.386500,2024-12-01-MS,2024-12-01,MS,ÓLEO DIESEL,0.000
4197,2024-12-01,MT,ÓLEO DIESEL,189891.719425,2024-12-01-MT,2024-12-01,MT,ÓLEO DIESEL,0.000
4198,2024-12-01,PA,ÓLEO DIESEL,163693.885025,2024-12-01-PA,2024-12-01,PA,ÓLEO DIESEL,0.000
4199,2024-12-01,PB,ÓLEO DIESEL,33616.031500,2024-12-01-PB,2024-12-01,PB,ÓLEO DIESEL,0.000
4200,2024-12-01,PE,ÓLEO DIESEL,99777.506575,2024-12-01-PE,2024-12-01,PE,GASOLINA A,2115.982
4201,2024-12-01,PI,ÓLEO DIESEL,40998.353425,2024-12-01-PI,2024-12-01,PI,ÓLEO DIESEL,0.000


In [ ]:
# demand_SanD.to_csv('[S&D_Demand_EthaGas].csv')
# Converte para uma lista de tuplas e cria o DataFrame
df_ethanol_blend = pd.DataFrame(
    list(ethanol_blend.items()),
    columns=['Ano', 'Percentual_Anidro']
)

print(df_ethanol_blend)

     Ano  Percentual_Anidro
0   2012               20.0
1   2013               25.0
2   2014               25.0
3   2015               27.0
4   2016               27.0
5   2017               27.0
6   2018               27.0
7   2019               27.0
8   2020               27.0
9   2021               27.0
10  2022               27.0
11  2023               27.5
12  2024               27.5


In [ ]:
supply_and_demand_SandD_gas.to_csv('[S&D_GASOLINE].csv')
df_ethanol_blend.to_csv('[S&D_dim_ethanol_blend].csv')